In [28]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [29]:
from tickers import getTickerList, getCandlePatternList
from datemodule import getDateRange
from indicators import ChannelBreakoutIndicator
from datasource import getFullData
import pandas

In [30]:
import talib
import pandas_ta as ta
from tqdm import tqdm

tickers = getTickerList(all=True)
tickerShortlist = {}
tickerShortlistData = {}

for i in tqdm(range(len(tickers))):
    ticker = tickers[i]
    resultDict = {}

    # Fetch Data
    startDate, endDate = getDateRange('3m')
    df = getFullData(ticker, startDate, endDate)
    
    candleIndex = len(df)-1
    
    # RSI
    df['RSI'] = ta.rsi(df.Close, length=14)

    # Candlestick Pattern Recognition
    op = df['Open']
    hi = df['High']
    lo = df['Low']
    cl = df['Close']
    for pattern in getCandlePatternList(all=False):
        sig = getattr(talib, pattern)(op, hi, lo, cl)[candleIndex]
        if sig != 0:
            resultDict[pattern] = sig
    
    # Channel Breakout Indicator Signal
    cbIndicator = ChannelBreakoutIndicator(df, ticker)
    cbIndicator.calculate(40)
    cbSignal = cbIndicator.getBuySell()[-1]
    if (cbSignal != ''):
        resultDict["ChannelBreakoutIndicator"] = cbSignal

    # Collect Signals
    if (len(resultDict) != 0):
        resultDict["RSI"] = df['RSI'][candleIndex]
        tickerShortlistData[ticker] = cbIndicator
        tickerShortlist[ticker] = resultDict

100%|██████████| 96/96 [00:50<00:00,  1.89it/s]


In [26]:
shortlisted = pandas.DataFrame(tickerShortlist).transpose()
entries = []
for index, row in shortlisted.iterrows():
    entries.append(row.count())
shortlisted["ENTRIES"] = entries
shortlisted = shortlisted.sort_values(by='ENTRIES', ascending=False)
shortlisted

,CDLSPINNINGTOP,ChannelBreakoutIndicator,RSI,CDLHARAMI,CDLDOJI,CDLEVENINGSTAR,CDLENGULFING,CDLMARUBOZU,CDLSHOOTINGSTAR,ENTRIES
BRITANNIA,100,SELL,64.485427,NaN,100,NaN,NaN,NaN,NaN,4
ADANIPORTS,100,SELL,54.598684,NaN,100,NaN,NaN,NaN,NaN,4
NAUKRI,-100.0,NaN,60.24478,100.0,100.0,NaN,NaN,NaN,NaN,4
ADANITRANS,100,SELL,41.941566,NaN,100,NaN,NaN,NaN,NaN,4
INDUSTOWER,100.0,NaN,53.80484,100.0,100.0,NaN,NaN,NaN,NaN,4
LT,100.0,NaN,69.797465,NaN,100.0,NaN,NaN,NaN,NaN,3
HAL,100.0,NaN,60.313233,NaN,100.0,NaN,NaN,NaN,NaN,3
LICI,100.0,NaN,61.65094,NaN,100.0,NaN,NaN,NaN,NaN,3
ATGL,-100.0,NaN,39.063262,NaN,100.0,NaN,NaN,NaN,NaN,3
ASIANPAINT,100.0,NaN,68.177922,NaN,100.0,NaN,NaN,NaN,NaN,3


In [27]:
for ticker in shortlisted.index:
    print(tickerShortlist[ticker])
    tickerShortlistData[ticker].showIndicator(len(tickerShortlistData[ticker].df.index)-1, 40)

{'CDLDOJI': 100, 'CDLSPINNINGTOP': 100, 'ChannelBreakoutIndicator': 'SELL', 'RSI': 64.48542662308921}
1.0 0.8653584599211888


{'CDLDOJI': 100, 'CDLSPINNINGTOP': 100, 'ChannelBreakoutIndicator': 'SELL', 'RSI': 54.59868434974787}
0.5926109860839412 0.16736523362750078


{'CDLDOJI': 100, 'CDLHARAMI': 100, 'CDLSPINNINGTOP': -100, 'RSI': 60.24477993614879}
0.9736975090579889 0.9913190068226787


{'CDLDOJI': 100, 'CDLSPINNINGTOP': 100, 'ChannelBreakoutIndicator': 'SELL', 'RSI': 41.94156617841782}
0.20709793404716734 1.0


{'CDLDOJI': 100, 'CDLHARAMI': 100, 'CDLSPINNINGTOP': 100, 'RSI': 53.80483982811744}
0.614322759327662 0.9405630042203784


...

{'ChannelBreakoutIndicator': 'BUY', 'RSI': 75.42910509211507}
1.0 0.9071902792922405


{'ChannelBreakoutIndicator': 'SELL', 'RSI': 58.19826995402091}
0.7976860697709935 0.908588842455045


{'CDLHARAMI': 100, 'RSI': 70.3405519662305}
1.0 1.0


{'ChannelBreakoutIndicator': 'BUY', 'RSI': 64.46197614245499}
1.0 1.0


{'CDLENGULFING': -100, 'RSI': 52.103007860909756}
1.0 0.6387279370437203
